# Py file testing

In [59]:
from ACF_GNR_estimation_functions import *

In [60]:
df_ACF = load_data("ACF")
df_GNR = load_data("GNR")

In [110]:
theta0 = np.array([1,1])/2
coeffs_ACF, convergence = ACF_estimation(df_ACF, theta0, print_results=1)

The gradient at the optimum is:  [-2.15671787e-15 -1.29091124e-15]
The GMM error using the gradient is: 4.558741784704597e-30
The estimates using autograd: [beta_k, beta_l] =  [0.3235676  0.73189068]
The slope of the AR(1) of productivity is: rho =  0.882158674944975


### Note: when bootstrapping, the rho falls. Why? It has to do with the time series properties of the data. 
### Use "block bootstrap" designed for panel data. 

In [116]:
df_boot_ACF = df_ACF.sample(frac=1, replace=True)
coeffs_ACF_boot, _ = ACF_estimation(df_boot_ACF, theta0, print_results=1)

The gradient at the optimum is:  [5.09085784e-13 3.27102621e-13]
The GMM error using the gradient is: 1.6647024463036653e-26
The estimates using autograd: [beta_k, beta_l] =  [0.29570628 0.77861083]
The slope of the AR(1) of productivity is: rho =  0.5549123580053624


In [8]:
alpha0 = np.ones(5)/2 #This is the required size to have coefficeints for k, l, kl, k**2, l**2. Need to change if the degree is 

results_params_GNR, results_convergence_GNR, alpha_GNR = GNR_estimation(df_GNR, alpha0, print_results = 1)

The error is: 1.7588529843286138e-23
The gradient is: [1.16328331e-10 7.79834445e-11 2.80147218e-09 1.55028140e-09
 8.06444239e-10]
The coefficients for the integration constant [alpha] are: [ 0.22962852 -2.6976986  -0.08259352  0.26112497  0.05193076]
The coefficients for productivity omega [delta] are: [1.97089299 0.17895714 0.08014284]
The average produc [1.97089299 0.17895714 0.08014284]
the average productivity [omega] is: 3.839691994977758
the average elasticity [df/dm] is: 0.5696607852773187


# Bootstrap:

In [ ]:
#Set seed

In [109]:
#Use the initial condition from the true data, coeffs_ACF[:2], to improve speed and convergence. 
bootstrap_results_ACF, convergence_ACF = bootstrap(ACF_estimation, coeffs_ACF[:2], df_ACF)
means_ACF = bootstrap_results_ACF.mean(axis=0)
std_devs_ACF = bootstrap_results_ACF.std(axis=0)
# Calculate percentiles along each column
lower_percentile_ACF = np.percentile(bootstrap_results_ACF, 2.5, axis=0)
upper_percentile_ACF = np.percentile(bootstrap_results_ACF, 97.5, axis=0)

In [12]:
#Use the initial condition from the true data, coeffs_ACF[:2], to improve speed and convergence. 
bootstrap_results_GNR, convergence_GNR = bootstrap(GNR_estimation, alpha_GNR, df_GNR)
means_GNR = bootstrap_results_GNR.mean(axis=0)
std_devs_GNR = bootstrap_results_GNR.std(axis=0)

KeyboardInterrupt: 